### 1. Collect the data

In [2]:
import pandas as pd

linkWiki="https://en.wikipedia.org/wiki/Democracy_Index"
democracy=pd.read_html(linkWiki, header=0,attrs={"class":"wikitable sortable"})[4]

linkmil="https://www.cia.gov/the-world-factbook/field/military-expenditures/country-comparison"

milimoney=pd.read_html(linkmil)[0]

linkHDI="https://github.com/UW-eScience-WinterSchool/Python_Session/raw/main/countryCodesHDI.xlsx"
hdidata=pd.read_excel(linkHDI)

### 2. Check column names

In [3]:
democracy.columns

Index(['Rank',
       '.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Δ Rank',
       'Country', 'Regime type', 'Overall score', 'Δ Score',
       'Elec­toral pro­cess and plura­lism', 'Func­tioning of govern­ment',
       'Poli­tical partici­pation', 'Poli­tical cul­ture', 'Civil liber­ties'],
      dtype='object')

In [4]:
milimoney.columns

Index(['Rank', 'Country', '% of GDP', 'Date of Information'], dtype='object')

In [5]:
hdidata.columns

Index(['FIPS', 'ISO2', 'ISO3', 'NAME', 'HDI'], dtype='object')

You checked column names to name the key columns for merge, and to get rid of columns that may bring trouble.

In [6]:
# this renaming will make merge easier
hdidata.rename(columns={'NAME':"Country"},inplace=True)

In [7]:
# merge will likely use "Rank" as the key (leftmost common variable), so get rid of it before it happens:
democracy.drop(columns=["Rank"],inplace=True)
milimoney.drop(columns=["Rank"],inplace=True)

### 3. Merge

In [8]:
allmerged=democracy.merge(milimoney)
allmerged=allmerged.merge(hdidata)
allmerged

,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Δ Rank,Country,Regime type,Overall score,Δ Score,Elec­toral pro­cess and plura­lism,Func­tioning of govern­ment,Poli­tical partici­pation,Poli­tical cul­ture,Civil liber­ties,% of GDP,Date of Information,FIPS,ISO2,ISO3,HDI
0,NaN,Norway,Full democracy,9.81,0.06,10.00,9.64,10.00,10.00,9.41,2.00,2020 est.,NO,NO,NOR,0.957
1,NaN,Sweden,Full democracy,9.26,0.13,9.58,9.29,8.33,10.00,9.12,1.20,2020,SW,SE,SWE,0.945
2,NaN,New Zealand,Full democracy,9.25,0.01,10.00,8.93,8.89,8.75,9.71,1.50,2020,NZ,NZ,NZL,0.931
3,2,Canada,Full democracy,9.24,0.02,9.58,8.93,8.89,9.38,9.41,1.39,2021 est.,CA,CA,CAN,0.929
4,1,Finland,Full democracy,9.20,0.05,10.00,8.93,8.89,8.75,9.41,2.20,2021 est.,FI,FI,FIN,0.938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,NaN,Tajikistan,Authoritarian,1.94,0.01,0.00,2.21,2.22,4.38,0.88,1.10,2020 est.,TI,TJ,TJK,0.668
139,1,Equatorial Guinea,Authoritarian,1.92,NaN,0.00,0.43,3.33,4.38,1.47,1.30,2019 est.,EK,GQ,GNQ,0.592
140,NaN,Turkmenistan,Authoritarian,1.72,NaN,0.00,0.79,2.22,5.00,0.59,0.90,2020 est.,TX,TM,TKM,0.715
141,NaN,Chad,Authoritarian,1.55,0.06,0.00,0.00,1.67,3.75,2.35,2.90,2020 est.,CD,TD,TCD,0.398


### 4. Preprocessing

* _Check the strings in column names_:

In [9]:
allmerged.columns.to_list()

['.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Δ Rank',
 'Country',
 'Regime type',
 'Overall score',
 'Δ Score',
 'Elec\xadtoral pro\xadcess and plura\xadlism',
 'Func\xadtioning of govern\xadment',
 'Poli\xadtical partici\xadpation',
 'Poli\xadtical cul\xadture',
 'Civil liber\xadties',
 '% of GDP',
 'Date of Information',
 'FIPS',
 'ISO2',
 'ISO3',
 'HDI']

* _Clean strings_:

In [10]:
#replace '%' by "share"
allmerged.columns=allmerged.columns.str.replace("\%","share",regex=True)
#replace 'spaces' by "_"
allmerged.columns=allmerged.columns.str.replace("\s","_",regex=True)
#replace 'whatever is not a character' by ""
allmerged.columns=allmerged.columns.str.replace("\W","",regex=True)
#current names
allmerged.columns.to_list()

['mwparseroutput_tooltipdottedborderbottom1px_dottedcursorhelpΔ_Rank',
 'Country',
 'Regime_type',
 'Overall_score',
 'Δ_Score',
 'Electoral_process_and_pluralism',
 'Functioning_of_government',
 'Political_participation',
 'Political_culture',
 'Civil_liberties',
 'share_of_GDP',
 'Date_of_Information',
 'FIPS',
 'ISO2',
 'ISO3',
 'HDI']

* _Drop MORE unneeded columns_:

In [11]:
#take a look:
allmerged.columns[allmerged.columns.str.contains("Rank|Date|Δ",regex=True)]

Index(['mwparseroutput_tooltipdottedborderbottom1px_dottedcursorhelpΔ_Rank',
       'Δ_Score', 'Date_of_Information'],
      dtype='object')

In [12]:
#save column to drop
toDrop=allmerged.columns[allmerged.columns.str.contains("Rank|Date|Δ",regex=True)]
# drop them
allmerged.drop(columns=toDrop,inplace=True)
# see result
allmerged

,Country,Regime_type,Overall_score,Electoral_process_and_pluralism,Functioning_of_government,Political_participation,Political_culture,Civil_liberties,share_of_GDP,FIPS,ISO2,ISO3,HDI
0,Norway,Full democracy,9.81,10.00,9.64,10.00,10.00,9.41,2.00,NO,NO,NOR,0.957
1,Sweden,Full democracy,9.26,9.58,9.29,8.33,10.00,9.12,1.20,SW,SE,SWE,0.945
2,New Zealand,Full democracy,9.25,10.00,8.93,8.89,8.75,9.71,1.50,NZ,NZ,NZL,0.931
3,Canada,Full democracy,9.24,9.58,8.93,8.89,9.38,9.41,1.39,CA,CA,CAN,0.929
4,Finland,Full democracy,9.20,10.00,8.93,8.89,8.75,9.41,2.20,FI,FI,FIN,0.938
...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,Tajikistan,Authoritarian,1.94,0.00,2.21,2.22,4.38,0.88,1.10,TI,TJ,TJK,0.668
139,Equatorial Guinea,Authoritarian,1.92,0.00,0.43,3.33,4.38,1.47,1.30,EK,GQ,GNQ,0.592
140,Turkmenistan,Authoritarian,1.72,0.00,0.79,2.22,5.00,0.59,0.90,TX,TM,TKM,0.715
141,Chad,Authoritarian,1.55,0.00,0.00,1.67,3.75,2.35,2.90,CD,TD,TCD,0.398


* _Look for missing values and check for wrong data types_:

In [13]:
allmerged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143 entries, 0 to 142
Data columns (total 13 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          143 non-null    object 
 1   Regime_type                      143 non-null    object 
 2   Overall_score                    143 non-null    object 
 3   Electoral_process_and_pluralism  143 non-null    object 
 4   Functioning_of_government        143 non-null    object 
 5   Political_participation          143 non-null    object 
 6   Political_culture                143 non-null    object 
 7   Civil_liberties                  143 non-null    object 
 8   share_of_GDP                     143 non-null    float64
 9   FIPS                             143 non-null    object 
 10  ISO2                             142 non-null    object 
 11  ISO3                             143 non-null    object 
 12  HDI                   

a. Some missing values can be corrected, others cannot:

In [14]:
allmerged[allmerged.isnull().any(axis=1)]

,Country,Regime_type,Overall_score,Electoral_process_and_pluralism,Functioning_of_government,Political_participation,Political_culture,Civil_liberties,share_of_GDP,FIPS,ISO2,ISO3,HDI
9,Taiwan,Full democracy,8.94,10.00,9.64,7.22,8.13,9.71,2.3,TW,TW,TWN,NaN
53,Namibia,Flawed democracy,6.52,7.00,5.36,6.67,5.63,7.94,3.4,WA,NaN,NAM,0.646


We can not use Taiwan, but Namibia can be kept.

In [15]:
allmerged.loc[pd.isnull(allmerged.ISO2),'ISO2']='NA'

Dropping rows with missing values:

In [16]:
allmerged.dropna(inplace=True)
allmerged.reset_index(drop=True,inplace=True)
allmerged

,Country,Regime_type,Overall_score,Electoral_process_and_pluralism,Functioning_of_government,Political_participation,Political_culture,Civil_liberties,share_of_GDP,FIPS,ISO2,ISO3,HDI
0,Norway,Full democracy,9.81,10.00,9.64,10.00,10.00,9.41,2.00,NO,NO,NOR,0.957
1,Sweden,Full democracy,9.26,9.58,9.29,8.33,10.00,9.12,1.20,SW,SE,SWE,0.945
2,New Zealand,Full democracy,9.25,10.00,8.93,8.89,8.75,9.71,1.50,NZ,NZ,NZL,0.931
3,Canada,Full democracy,9.24,9.58,8.93,8.89,9.38,9.41,1.39,CA,CA,CAN,0.929
4,Finland,Full democracy,9.20,10.00,8.93,8.89,8.75,9.41,2.20,FI,FI,FIN,0.938
...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,Tajikistan,Authoritarian,1.94,0.00,2.21,2.22,4.38,0.88,1.10,TI,TJ,TJK,0.668
138,Equatorial Guinea,Authoritarian,1.92,0.00,0.43,3.33,4.38,1.47,1.30,EK,GQ,GNQ,0.592
139,Turkmenistan,Authoritarian,1.72,0.00,0.79,2.22,5.00,0.59,0.90,TX,TM,TKM,0.715
140,Chad,Authoritarian,1.55,0.00,0.00,1.67,3.75,2.35,2.90,CD,TD,TCD,0.398


b. Convert string to numerical

In [17]:
toNumeric=['Overall_score',
 'Electoral_process_and_pluralism', 
 'Functioning_of_government',
 'Political_participation', 'Political_culture', 'Civil_liberties']
allmerged.loc[:,toNumeric]=allmerged.loc[:,toNumeric].apply(lambda x: pd.to_numeric(x))

c. Check ordinal variable:

In [18]:
#check levels:
set(allmerged.Regime_type)

{'Authoritarian', 'Flawed democracy', 'Full democracy', 'Hybrid regime'}

In [19]:
levels=['Authoritarian', 'Hybrid regime','Flawed democracy', 'Full democracy']
allmerged.Regime_type=pd.Categorical(allmerged.Regime_type,
                                     categories=levels,
                                     ordered=True)

In [20]:
allmerged.Regime_type

0      Full democracy
1      Full democracy
2      Full democracy
3      Full democracy
4      Full democracy
            ...      
137     Authoritarian
138     Authoritarian
139     Authoritarian
140     Authoritarian
141     Authoritarian
Name: Regime_type, Length: 142, dtype: category
Categories (4, object): ['Authoritarian' < 'Hybrid regime' < 'Flawed democracy' < 'Full democracy']

In [21]:
# review:
allmerged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 13 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   Country                          142 non-null    object  
 1   Regime_type                      142 non-null    category
 2   Overall_score                    142 non-null    float64 
 3   Electoral_process_and_pluralism  142 non-null    float64 
 4   Functioning_of_government        142 non-null    float64 
 5   Political_participation          142 non-null    float64 
 6   Political_culture                142 non-null    float64 
 7   Civil_liberties                  142 non-null    float64 
 8   share_of_GDP                     142 non-null    float64 
 9   FIPS                             142 non-null    object  
 10  ISO2                             142 non-null    object  
 11  ISO3                             142 non-null    object  
 12  HDI     

Let's give better names to the main indices:

In [23]:
LastNamesChanges={'Overall_score':'DemoIndex','share_of_GDP':'DefenseIndex','HDI':"HDIndex"}
allmerged.rename(columns=LastNamesChanges,inplace=True)

### 4. Save your work

* _Save data for R_:

In [24]:
from rpy2.robjects import pandas2ri
pandas2ri.activate()

from rpy2.robjects.packages import importr

base = importr('base')
base.saveRDS(allmerged,file="allmerged.rds")

/Users/JoseManuel/anaconda3/envs/WinterSchool/lib/python3.8/site-packages/rpy2/robjects/vectors.py:980: UserWarning: R object inheriting from "POSIXct" but without attribute "tzone".
  warnings.warn('R object inheriting from "POSIXct" but without '


<rpy2.rinterface_lib.sexp.NULLType object at 0x7fefffe9b4c0> [RTYPES.NILSXP]